## Originals in:
### https://www.kaggle.com/matleonard/text-classification

In [1]:
import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
spam.rename({"v1" : "label", "v2" : "text"}, axis=1, inplace=True)
spam.head(10)

,label,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [2]:
spam.shape

(5572, 5)

In [3]:
spam.label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

### Bag of Words

Machine learning models don't learn from raw text data. Instead, you need to convert the text to something numeric.

---

The simplest common representation is a **variation of one-hot encoding**. You represent each document as a vector of term frequencies for each term in the vocabulary. The vocabulary is built from all the tokens (terms) in the **corpus (the collection of documents)**.

---

As an example, take the sentences "Tea is life. Tea is love." and "Tea is healthy, calming, and delicious." as our corpus. The vocabulary then is ```{"tea", "is", "life", "love", "healthy", "calming", "and", "delicious"}``` (ignoring punctuation).

---

For each document, count up how many times a term occurs, and place that count in the appropriate element of a vector. The first sentence has "tea" twice and that is the first position in our vocabulary, so we put the number 2 in the first element of the vector. Our sentences as vectors then look like

---

v1 = [2 2 1 1 0 0 0 0]   
v2 = [1 1 0 0 1 1 1 1]

---

This is called the **bag of words** representation. You can see that documents with similar terms will have similar vectors. Vocabularies frequently have tens of thousands of terms, so these vectors can be very large.

---

Another common representation is **TF-IDF (Term Frequency - Inverse Document Frequency)**. TF-IDF is similar to bag of words except that each term count is scaled by the term's frequency in the corpus. Using TF-IDF can potentially improve your models. You won't need it here. Feel free to look it up though!

### Building a Bag of Words model

Once you have your documents in a bag of words representation, you can use those vectors as input to any machine learning model. spaCy handles the bag of words conversion and building a simple linear model for you with the ```TextCategorizer``` class.

---

The TextCategorizer is a spaCy **pipe**. Pipes are classes for processing and transforming tokens. When you create a spaCy model with ```nlp = spacy.load('en_core_web_sm')```, there are default pipes that perform part of speech tagging, entity recognition, and other transformations. When you run text through a model ```doc = nlp("Some text here")```, the output of the pipes are attached to the tokens in the doc object. The lemmas for ```token.lemma_``` come from one of these pipes.

---

You can remove or add pipes to models. What we'll do here is create an empty model without any pipes (other than a tokenizer, since all models always have a tokenizer). Then, we'll create a TextCategorizer pipe and add it to the empty model.

In [4]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
# "textcat" stands for TextCategorizer
# "bow" stands for Bag of Words
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

Since the classes are either ham or spam, we set ```"exclusive_classes"``` to True. We've also configured it with the **bag of words ("bow")** architecture. spaCy provides a convolutional neural network architecture as well, but it's more complex than you need for now.

Next we'll add the labels to the model. Here "ham" are for the real messages, "spam" are spam messages. (Esse passo nao deveria ser antes do nlp.add_pipe(textcat) ? )

In [5]:
# Add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

### Training a Text Categorizer Model

Next, you'll convert the labels in the data to the form TextCategorizer requires. For each document, you'll create a dictionary of boolean values for each class.

For example, if a text is "ham", we need a dictionary ```{'ham': True, 'spam': False}```. The model is looking for these labels inside another dictionary with the key ```'cats'```.

In [6]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]
train_labels[:3]

[{'cats': {'ham': True, 'spam': False}},
 {'cats': {'ham': True, 'spam': False}},
 {'cats': {'ham': False, 'spam': True}}]

Then we combine the texts and labels into a single list.

In [7]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

Now you are ready to train the model. First, create an ```optimizer``` using ```nlp.begin_training()```. spaCy uses this optimizer to update the model. In general it's more efficient to train models in small batches. spaCy provides the minibatch function that returns a generator yielding minibatches for training. Finally, the minibatches are split into texts and labels, then used with ```nlp.update``` to update the model's parameters.

In [8]:
from spacy.util import minibatch
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 1.3391926079602854}
{'textcat': 1.6745406250865926}
{'textcat': 1.8633504289379772}
{'textcat': 1.9841395507194513}
{'textcat': 2.0651512457259322}
{'textcat': 2.11714940062655}
{'textcat': 2.153624967674375}
{'textcat': 2.178099393301986}
{'textcat': 2.196800914543931}
{'textcat': 2.2107666343862644}


### Making Predictions

Now that you have a trained model, you can make predictions with the ```predict()``` method. The input text needs to be tokenized with ```nlp.tokenizer```. Then you pass the tokens to the predict method which returns scores. The scores are the probability the input text belongs to the classes.

In [9]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.9992990e-01 7.0148832e-05]
 [1.8173415e-02 9.8182666e-01]]


The scores are used to predict a single class or label by choosing the label with the highest probability. You get the index of the highest probability with ```scores.argmax```, then use the index to get the label string from ```textcat.labels```.

In [10]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'spam']


Evaluating the model is straightforward once you have the predictions. To measure the accuracy, calculate how many correct predictions are made on some test data, divided by the total number of predictions.